# Basic Setup

In [1]:
import requests as req
from bs4 import BeautifulSoup as BS

headers = {}
payload = {}

res = req.get('https://www.ptt.cc/bbs/Boy-Girl/index.html')
res = req.post('URL',data=payload,headers=headers)
html = BS(res.text,'html.parser')

for all in html.find('span',{'class':'hl f1'}):
    print(all.parent.parent.parent.a.contents) # Title
    print(all.parent.parent.parent.a['href']) # URN


['[公告] 即日起停止匿名信箱服務。']
/bbs/Boy-Girl/M.1399018430.A.4CF.html


# Selenium

In [13]:
import time
from selenium import webdriver
driver = webdriver.Chrome() # Chrome, PhantomJS, etc. 
driver.get('https://www.ptt.cc/bbs/Boy-Girl/index.html')

target = " 上頁"
for i in range(0,3):
    time.sleep(1.5)
    lastP = driver.find_element_by_partial_link_text(target).get_attribute("href")
    driver.get(lastP)
    driver.save_screenshot(str(i) + '.png')

# How to Solve SSL1 Error : crawl NTU courses

In [ ]:
import requests, time
from bs4 import BeautifulSoup as BS
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
import ssl

class MyAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = PoolManager(num_pools=connections,
                                       maxsize=maxsize,
                                       block=block,
                                       ssl_version=ssl.PROTOCOL_TLSv1)
        
rownum = 0
with requests.Session() as s:
    res = s.mount('https://',MyAdapter())
    res = s.get('https://nol.ntu.edu.tw/nol/guest/index.php')
    for i in range(0,12500,500):
        time.sleep(1)
        print(i, end='\t')
        payload = {'alltime':'yes',
                   'allproced':'yes',
                   'cstype':'1',
                   'current_sem':'106-1',
                   'op':'stu',
                   'startrec':str(i),
                   'allsel':'yes',
                   'page_cnt':'500'}
        res = s.post('https://nol.ntu.edu.tw/nol/coursesearch/search_result.php',data=payload)
        res.encoding = 'cp950'
        soup = BS(res.text,'html.parser')

        with open('NTUCourses.txt','a+',encoding='utf8') as f:
            for tr in soup.select('table')[6].select('tr'):
                rownum+=1
                for td in tr.select('td'):
                    f.write(td.text.replace("\t","").replace("\n","").replace("\r","").strip() + '\t')
                f.write('\n')
        print('Done')
print(rownum)

# Joint Credit Information Center

In [ ]:
import requests, sys, re, time
import pandas as pd
from bs4 import BeautifulSoup as BS
pause = 0.5

params = {'#ContentPlaceHolder1_F_BUILDING_CITY':'台北市',
          '#ContentPlaceHolder1_F_BUILDING_CLASS':'電梯大廈',
          '#ContentPlaceHolder1_F_PERSON_EDUCATION':'大學'}
county = '中山區'

payload = {'ctl00$ContentPlaceHolder1$ScriptManager1':'ctl00$ContentPlaceHolder1$ScriptManager1|ctl00$ContentPlaceHolder1$F_BUILDING_CITY',
           '__EVENTTARGET':'ctl00$ContentPlaceHolder1$F_BUILDING_CITY',
           'ctl00$tb_keyword':'請輸入關鍵字',
           'ctl00$ContentPlaceHolder1$F_DATA_TYPE':'1',
           'ContentPlaceHolder1_Tabs_ClientState':'{"ActiveTabIndex":0,"TabState":[true,true,true]}',
           'ctl00$ContentPlaceHolder1$F_DATA_PERIOD_S':'2008Q2', #資料期間
           'ctl00$ContentPlaceHolder1$F_DATA_PERIOD_E':'2017Q1', #資料期間
           'ctl00$ContentPlaceHolder1$F_STATISTICS':'1',
           'ctl00$ContentPlaceHolder1$F_BUILDING_COUNTY':'0000000', # 行政區別
           'ctl00$ContentPlaceHolder1$F_BUILDING_ROAD':'請先選取縣市別及行政區別', #路/街 
           'ctl00$ContentPlaceHolder1$F_BUILDING_SECTION':'0', #段
           'ctl00$ContentPlaceHolder1$F_BUILDING_CLASS':'02', #建物類別
           'ctl00$ContentPlaceHolder1$F_BUILDING_AGE_S':'00', #屋齡區間
           'ctl00$ContentPlaceHolder1$F_BUILDING_AGE_E':'36', #屋齡區間
           'ctl00$ContentPlaceHolder1$F_PERSON_GENDER':'1', #性別
           'ctl00$ContentPlaceHolder1$F_PERSON_AGE_S':'20', #年齡區間
           'ctl00$ContentPlaceHolder1$F_PERSON_AGE_E':'80', #年齡區間
           'ctl00$ContentPlaceHolder1$F_PERSON_INCOME_S':'000', #年收入區間
           'ctl00$ContentPlaceHolder1$F_PERSON_INCOME_E':'400', #年收入區間
           'ctl00$ContentPlaceHolder1$F_PERSON_EDUCATION':'4'} #教育程度
"""
headers = {'X-Requested-With':'XMLHttpRequest',
           'Origin':'https://member.jcic.org.tw',
           'Referer':'https://member.jcic.org.tw/main_member/MorgageQuery.aspx',
           'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}                 
"""

with requests.session() as s:
    time.sleep(pause)
    r = s.get('https://member.jcic.org.tw/main_member/MorgageQuery.aspx')
    html = BS(r.content, 'html.parser')
    for ID, param in params.items():
        options = []
        for option in html.select(ID)[0].select('option'):
            options.append(option.text)
            if option.text == param:
                print(option.text,'代碼 :',option['value'])
                payload[html.select(ID)[0]['name']] = option['value']
                break
        else:
            print('選項列表提示 :',options)
            sys.exit(param + ' 可能不存在於選項列表中，也可能是標籤的id發生更動!')
    payload['__VIEWSTATE'] = html.select('#__VIEWSTATE')[0]['value']
    payload['__VIEWSTATEGENERATOR'] = html.select('#__VIEWSTATEGENERATOR')[0]['value']
    
    time.sleep(pause)
    r_city = s.post('https://member.jcic.org.tw/main_member/MorgageQuery.aspx', data=payload) #取得行政區別資料
    html = BS(r_city.content, 'html.parser')        
    options = []
    for option in html.select('#ContentPlaceHolder1_F_BUILDING_COUNTY')[0].select('option'):
        options.append(option.text)
        if option.text == county:
            print(option.text,'代碼 :',option['value'])
            payload['ctl00$ContentPlaceHolder1$F_BUILDING_COUNTY'] = option['value']
            break
    else:
        print('選項列表提示 :',options)
        sys.exit(county + ' 可能不存在於選項列表中，也可能是標籤的id發生更動!')
    payload['ctl00$ContentPlaceHolder1$ScriptManager1'] = 'ctl00$ContentPlaceHolder1$ScriptManager1|ctl00$ContentPlaceHolder1$F_BUILDING_COUNTY'
    payload['__EVENTTARGET'] = 'ctl00$ContentPlaceHolder1$F_BUILDING_COUNTY'
    payload['__VIEWSTATE'] = html.select('#__VIEWSTATE')[0]['value']  
    
    time.sleep(pause)
    r_county = s.post('https://member.jcic.org.tw/main_member/MorgageQuery.aspx', data=payload)
    html = BS(r_county.content,'html.parser')
    payload['__VIEWSTATE'] = html.select('#__VIEWSTATE')[0]['value']
    payload['ctl00$ContentPlaceHolder1$ScriptManager1'] = 'ctl00$ContentPlaceHolder1$ScriptManager1|ctl00$ContentPlaceHolder1$bt_Query'
    payload['ContentPlaceHolder1_Tabs_Panel_Chart_Tab_DATA_ClientState'] = '{"ActiveTabIndex":0,"TabState":[true,true,true,true,true,true,true,true]}'
    payload['ContentPlaceHolder1_Tabs_TabPanel1_Tab_PIE_ClientState'] = '{"ActiveTabIndex":0,"TabState":[true,true,true,true,true,true,true]}'
    payload['ctl00$ContentPlaceHolder1$bt_Query'] = '查詢'
    del payload['__EVENTTARGET']
    del payload['ctl00$ContentPlaceHolder1$F_BUILDING_ROAD']
    
    time.sleep(pause)
    r_data = s.post('https://member.jcic.org.tw/main_member/MorgageQuery.aspx', data=payload)
    html = BS(r_data.content,'html.parser')
    for tr in html.select('#ContentPlaceHolder1_Tabs_Panel_Table_gvPaging')[0].select('tr'):
        for td in tr.select('td'):
            print(td.text.strip(),end='\t')
        print()